<h1>Preprocessing </h1>

In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os
import sys # for debugging 

In [3]:

# # This function preprocesses the image by reading in the image apply grayscale make all the sizes the same and 
# def preprocess_image(file_path, img_size):
#     img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) # Grayscale will even the playing field if we start getting different types of images. If the images color is a factor we can take out grayscale
#     img = cv2.resize(img, img_size)
#     img = img.astype('float')/255.0 # Make the pixels become float and normalize to 0-1 for normalization
#     return img







# This function preprocesses the image by reading in the image apply grayscale make all the sizes the same and 
def preprocess_image(file_path, img_size):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) # Grayscale will even the playing field if we start getting different types of images. If the images color is a factor we can take out grayscale
    
    # Thresholding to remove black background
    _, binary_image = cv2.threshold(img, 10, 255, cv2.THRESH_BINARY)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_image, connectivity=8)
    largest_component_label = np.argmax(stats[1:, cv2.CC_STAT_AREA]) + 1
    brain_mask = (labels == largest_component_label).astype(np.uint8) * 255
    x, y, w, h = cv2.boundingRect(brain_mask)
    img = img[y:y+h, x:x+w]
    
    img = cv2.resize(img, img_size)
    img = img.astype('float')/255.0 # Make the pixels become float and normalize to 0-1 for normalization
    return img


target_size =(224, 224)

# This function will pull from the directory and all subdirectory for the image and give it a label to the directory it is in
def load_images_from_directory(directory):
    images = []
    labels = []
    # Iterates through all subdirectories
    for subdir in os.listdir(directory):
        label = subdir #Make the subdirectory name be a label
        subdir_path = os.path.join(directory, subdir)

        # Checks if the object it is looking at is a directory and if it is go into the directory and get all the files and preprocess them
        if os.path.isdir(subdir_path):
            for image in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, image)

                image = preprocess_image(file_path, target_size)

                # Append to the arrays after preprocessing
                images.append(image)
                labels.append(label)

    return np.array(images), np.array(labels)




In [4]:


# Define the directory paths for the training and test datasets
train_dir = "./Alzheimer_s Dataset/train"
test_dir = "./Alzheimer_s Dataset/test"
# single_test_dir = "./Alzheimer_s Dataset/single_test"

# Load images and labels from the training directory
alz_images_train, alz_labels_train = load_images_from_directory(train_dir)

# Load images and labels from the test directory
alz_images_test, alz_labels_test = load_images_from_directory(test_dir)

# alz_single_images_test, alz_single_labels_test = load_images_from_directory(single_test_dir)

# Print information about the training dataset
print("Train")
print('Image shape:', alz_images_train.shape)
print('Labels shape:', alz_labels_train.shape)

# Print information about the test dataset
print("\nTest")
print('Image shape:', alz_images_test.shape)
print('Labels shape:', alz_labels_test.shape)


# print("\nSingle Test")
# print('Image shape:', alz_single_images_test.shape)
# print('Labels shape:', alz_single_labels_test.shape)


# np.set_printoptions(threshold=sys.maxsize) # for debugging

# print('Image train:', alz_single_images_test) # for debugging

# The output of the shape follows this
#  (X, X1, X2)
# X is the number of pictures in the array   
# X1 is the number of rows for a single picture (should be 224 since that is the scale)
# X2 is the number of columns in each picture  (should be 224 since that is the scale)
#  *Scale can be change to 207 since that is how the data is processed. 
# 
# When pull out the full array, you see alot of 0 at the start and end and that is because of the black around the brain
# 


Train
Image shape: (5121, 224, 224)
Labels shape: (5121,)

Test
Image shape: (1279, 224, 224)
Labels shape: (1279,)


<h1> Aaron's Algorithm </h1>
CNN GCNN or similar neural networks that can be adjusted in between each other

In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

alz_labels_train_encoded = label_encoder.fit_transform(alz_labels_train)
alz_labels_test_encoded = label_encoder.fit_transform(alz_labels_test)

num_classes = len(label_encoder.classes_)

alz_labels_train_onehot = tf.keras.utils.to_categorical(alz_labels_train_encoded, num_classes)
alz_labels_test_onehot = tf.keras.utils.to_categorical(alz_labels_test_encoded, num_classes)

#np.set_printoptions(threshold=sys.maxsize) # for debugging
#print(alz_labels_train_onehot)

print("Training labels shape (one-hot encoded):", alz_labels_train_onehot.shape)
print("Testing labels shape (one-hot encoded):", alz_labels_test_onehot.shape)

# print('Image train:', alz_images_train) # for debugging


# 0 = MildDemented
# 1 = ModerateDemented
# 2 = NonDemented
# 3 = VeryMildDemented

Training labels shape (one-hot encoded): (5121, 4)
Testing labels shape (one-hot encoded): (1279, 4)


In [6]:
img_height = target_size[1]
img_width = target_size[0]
num_channels = 1

<h1> CNN </h1>

In [39]:
#3RD BEST MODEL
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define model architecture with batch normalization
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, num_channels), 
           kernel_regularizer=regularizers.l2(0.1)),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model with Adam optimizer and categorical crossentropy loss
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with early stopping and learning rate scheduler
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
learning_rate_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

history = model.fit(alz_images_train, alz_labels_train_onehot, epochs=30, batch_size=32,
                     validation_data=(alz_images_test, alz_labels_test_onehot), 
                     callbacks=[early_stopping, learning_rate_scheduler])


Epoch 1/30
161/161 [==============================] - 91s 556ms/step - loss: 27.6525 - accuracy: 0.5438 - val_loss: 23.6364 - val_accuracy: 0.5004 - lr: 0.0010
Epoch 2/30
161/161 [==============================] - 89s 553ms/step - loss: 17.0131 - accuracy: 0.6259 - val_loss: 29.0601 - val_accuracy: 0.3503 - lr: 0.0010
Epoch 3/30
161/161 [==============================] - 89s 554ms/step - loss: 15.9307 - accuracy: 0.6251 - val_loss: 18.6328 - val_accuracy: 0.5004 - lr: 0.0010
Epoch 4/30
161/161 [==============================] - 89s 553ms/step - loss: 12.1645 - accuracy: 0.6272 - val_loss: 11.2882 - val_accuracy: 0.5004 - lr: 0.0010
Epoch 5/30
161/161 [==============================] - 89s 551ms/step - loss: 10.0402 - accuracy: 0.6532 - val_loss: 10.6785 - val_accuracy: 0.3511 - lr: 0.0010
Epoch 6/30
161/161 [==============================] - 89s 554ms/step - loss: 9.9922 - accuracy: 0.6585 - val_loss: 8.6916 - val_accuracy: 0.5059 - lr: 0.0010
Epoch 7/30
161/161 [======================

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping

# THE 2ND BEST
# Learning rate scheduler - Exponential Decay
def exponential_decay(epoch, initial_lr=0.001, decay_rate=0.9):
    return initial_lr * np.power(decay_rate, epoch)

#THE BEST
# Learning rate scheduler - Cyclic Learning Rate
def cyclic_lr(epoch, lr_max=0.001, lr_min=0.0001, step_size=8):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = lr_min + (lr_max - lr_min) * np.maximum(0, (1 - x))
    return lr

# Define model architecture with batch normalization
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, num_channels), 
           kernel_regularizer=regularizers.l2(0.1)),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.1), kernel_initializer=he_normal()),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model with Adam optimizer and categorical crossentropy loss
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with early stopping and learning rate scheduler
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# learning_rate_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

lr_scheduler_exp_decay = LearningRateScheduler(exponential_decay)
lr_scheduler_cyclic_lr = LearningRateScheduler(cyclic_lr)

# history = model.fit(alz_images_train, alz_labels_train_onehot, epochs=30, batch_size=32,
#                      validation_data=(alz_images_test, alz_labels_test_onehot), 
#                      callbacks=[early_stopping, learning_rate_scheduler])

history_exp_decay = model.fit(alz_images_train, alz_labels_train_onehot, epochs=30, batch_size=32,
                              validation_data=(alz_images_test, alz_labels_test_onehot), 
                              callbacks=[lr_scheduler_exp_decay, early_stopping])

history_cyclic_lr = model.fit(alz_images_train, alz_labels_train_onehot, epochs=30, batch_size=32,
                              validation_data=(alz_images_test, alz_labels_test_onehot), 
                              callbacks=[lr_scheduler_cyclic_lr, early_stopping])

Epoch 1/30
161/161 [==============================] - 92s 565ms/step - loss: 27.6656 - accuracy: 0.5167 - val_loss: 20.6026 - val_accuracy: 0.1400 - lr: 0.0010
Epoch 2/30
161/161 [==============================] - 91s 563ms/step - loss: 17.7674 - accuracy: 0.5536 - val_loss: 21.7103 - val_accuracy: 0.3503 - lr: 9.0000e-04
Epoch 3/30
161/161 [==============================] - 91s 566ms/step - loss: 12.3870 - accuracy: 0.6091 - val_loss: 9.8986 - val_accuracy: 0.3503 - lr: 7.2900e-04
Epoch 4/30
161/161 [==============================] - 91s 564ms/step - loss: 8.4417 - accuracy: 0.6516 - val_loss: 8.3055 - val_accuracy: 0.3503 - lr: 5.3144e-04
Epoch 5/30
161/161 [==============================] - 91s 563ms/step - loss: 4.7702 - accuracy: 0.7389 - val_loss: 4.7302 - val_accuracy: 0.3729 - lr: 3.4868e-04
Epoch 6/30
161/161 [==============================] - 91s 563ms/step - loss: 2.4933 - accuracy: 0.8377 - val_loss: 2.8808 - val_accuracy: 0.5395 - lr: 2.0589e-04
Epoch 7/30
161/161 [=======

<h1> DNN </h1>

In [7]:
# Reshape the input data to have rank 4
alz_images_train_dnn = alz_images_train.reshape(-1, 224, 224, 1)
alz_images_test_dnn = alz_images_test.reshape(-1, 224, 224, 1)

# Verify the shapes
print("Training data shape:", alz_images_train.shape)
print("Testing data shape:", alz_images_test.shape)


Training data shape: (5121, 224, 224)
Testing data shape: (1279, 224, 224)


In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define model architecture with batch normalization
model = Sequential([
    Flatten(input_shape=(img_height, img_width, num_channels)),

    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),

    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    
    Dropout(0.2),

    Dense(num_classes, activation='softmax')
])

# Compile the model with Adam optimizer and categorical crossentropy loss
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks for early stopping and learning rate scheduler
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def exponential_decay(epoch, initial_lr=0.001, decay_rate=0.9):
    return initial_lr * np.power(decay_rate, epoch)

lr_scheduler_exp_decay = LearningRateScheduler(exponential_decay)


# Create an instance of ImageDataGenerator with desired augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,      # Randomly rotate images by up to 20 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally by up to 10% of the width
    height_shift_range=0.1, # Randomly shift images vertically by up to 10% of the height
    shear_range=0.2,        # Randomly apply shear transformations
    zoom_range=0.2,         # Randomly zoom in by up to 20%
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest'     # Fill in newly created pixels (due to augmentation) using the nearest existing pixel
)


# Define batch size and number of epochs
batch_size = 32
epochs = 30

# Create augmented training data generator
train_generator = datagen.flow(alz_images_train_dnn, alz_labels_train_onehot, batch_size=batch_size)

# Train the model using the augmented data generator
history = model.fit(train_generator,
                    steps_per_epoch=len(alz_images_train) // batch_size,
                    epochs=epochs,
                    validation_data=(alz_images_test_dnn, alz_labels_test_onehot),
                    callbacks=[early_stopping, lr_scheduler_exp_decay])



Epoch 1/30
160/160 [==============================] - 27s 166ms/step - loss: 8.8108 - accuracy: 0.4496 - val_loss: 3.7299 - val_accuracy: 0.3862 - lr: 0.0010
Epoch 2/30
160/160 [==============================] - 26s 165ms/step - loss: 2.9792 - accuracy: 0.4826 - val_loss: 3.0124 - val_accuracy: 0.5223 - lr: 9.0000e-04
Epoch 3/30
160/160 [==============================] - 27s 166ms/step - loss: 2.0992 - accuracy: 0.4934 - val_loss: 1.9373 - val_accuracy: 0.3315 - lr: 7.2900e-04
Epoch 4/30
160/160 [==============================] - 26s 162ms/step - loss: 1.6142 - accuracy: 0.5044 - val_loss: 1.4650 - val_accuracy: 0.5317 - lr: 5.3144e-04
Epoch 5/30
160/160 [==============================] - 26s 165ms/step - loss: 1.4068 - accuracy: 0.4926 - val_loss: 1.3298 - val_accuracy: 0.5090 - lr: 3.4868e-04
Epoch 6/30
160/160 [==============================] - 26s 163ms/step - loss: 1.2846 - accuracy: 0.5060 - val_loss: 1.2295 - val_accuracy: 0.4980 - lr: 2.0589e-04
Epoch 7/30
160/160 [============

<h1>CNN with Graph based features</h1>


In [8]:
from sklearn.decomposition import PCA

# Generator function to yield batches of preprocessed images
def image_generator(images, batch_size=32):
    num_images = len(images)
    num_batches = (num_images + batch_size - 1) // batch_size
    
    for i in range(num_batches):
        start_index = i * batch_size
        end_index = min((i + 1) * batch_size, num_images)
        
        # Load and preprocess images for the current batch
        batch_images = np.stack((images[start_index:end_index],) * 3, axis=-1)
        preprocessed_images = tf.keras.applications.mobilenet_v2.preprocess_input(batch_images)
        
        yield preprocessed_images

# Function to extract features from images using a pre-trained CNN and perform PCA
def extract_features(images, batch_size=32, n_components=64):
    feature_extractor = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
    feature_extractor.trainable = False
    
    features = []
    for batch_images in image_generator(images, batch_size=batch_size):
        batch_features = feature_extractor.predict(batch_images)
        batch_features_flat = batch_features.reshape(batch_features.shape[0], -1)
        features.append(batch_features_flat)
    
    all_features = np.concatenate(features, axis=0)
    
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(all_features)
    
    return reduced_features

# Extract features from training and test images
train_features = extract_features(alz_images_train)
test_features = extract_features(alz_images_test)

1/1 [==============================] - 1s 713ms/step


In [9]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Compute adjacency matrix based on feature similarity
def compute_feature_similarity(features):
    num_images = features.shape[0]
    similarities = np.zeros((num_images, num_images))
    for i in range(num_images):
        for j in range(num_images):
            # Compute cosine similarity between feature vectors
            similarities[i, j] = cosine_similarity(features[i].reshape(1, -1), features[j].reshape(1, -1))[0, 0]
    return similarities

In [11]:
# Compute feature similarities for training and test images
train_feature_similarity = compute_feature_similarity(train_features)

In [10]:
test_feature_similarity = compute_feature_similarity(test_features)

In [12]:
similarity_threshold = 0.8

# Construct adjacency matrix based on feature similarity
def construct_adjacency_matrix(feature_similarity, threshold):
    num_images = feature_similarity.shape[0]
    adjacency_matrix = np.zeros((num_images, num_images))
    for i in range(num_images):
        for j in range(num_images):
            # Set adjacency matrix value based on whether feature similarity is above threshold
            if feature_similarity[i, j] >= threshold:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1  # Symmetric adjacency matrix
    return adjacency_matrix


In [13]:
# Compute adjacency matrix for training and test images
train_adj_matrix = construct_adjacency_matrix(train_feature_similarity, similarity_threshold)
print("Shape of adjacency matrix for training images:", train_adj_matrix.shape)

Shape of adjacency matrix for training images: (5121, 5121)


In [14]:
test_adj_matrix = construct_adjacency_matrix(test_feature_similarity, similarity_threshold)
print("Shape of adjacency matrix for test images:", test_adj_matrix.shape)

Shape of adjacency matrix for test images: (1279, 1279)


In [30]:
print("Shape of train_features:", train_features.shape)
print("Shape of test_features:", test_features.shape)

print("\nShape of train_features similarity:", train_feature_similarity.shape)
print("Shape of test_features similarity:", test_feature_similarity.shape)

print("\nShape of train_adj_matrix:", train_adj_matrix.shape)
print("Shape of test_adj_matrix:", test_adj_matrix.shape)


Shape of train_features: (5121, 64)
Shape of test_features: (1279, 64)

Shape of train_features similarity: (5121, 5121)
Shape of test_features similarity: (1279, 1279)

Shape of train_adj_matrix: (5121, 5121)
Shape of test_adj_matrix: (1279, 1279)


In [16]:
# Images with grayscale index
alz_images_train_with_grayscale_index = alz_images_train[..., np.newaxis]  # Add channel dimension for grayscale images
alz_images_test_with_grayscale_index = alz_images_test[..., np.newaxis]

# Integrate graph-based features
train_features_with_graph = np.concatenate([train_features, train_adj_matrix], axis=1)
test_features_with_graph = np.concatenate([test_features, test_adj_matrix], axis=1)

print("Shape of training images with grayscale index:", alz_images_train_with_grayscale_index.shape)
print("Shape of testing images with grayscale index:", alz_images_test_with_grayscale_index.shape)
print("Shape of training features with graph:", train_features_with_graph.shape)
print("Shape of testing features with graph:", test_features_with_graph.shape)

Shape of training images with grayscale index: (5121, 224, 224, 1)
Shape of testing images with grayscale index: (1279, 224, 224, 1)
Shape of training features with graph: (5121, 5185)
Shape of testing features with graph: (1279, 1343)


In [40]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Flatten, Dense

# Define input layers for image data and graph data
image_input = Input(shape=(img_height, img_width, 1), name='image_input')
graph_input = Input(shape=(train_features.shape[1],), name='graph_input')
adj_input = Input(shape=(train_adj_matrix.shape[1],), name='adj_input')

# Flatten the image data
flatten_image = Flatten()(image_input)

# Concatenate flattened image data with graph data and adjacency matrix
concatenated_input = Concatenate()([flatten_image, graph_input, adj_input])

# Define the dense layers
x = Dense(128, activation='relu')(concatenated_input)
x = Dense(64, activation='relu')(x)

# Output layer
output = Dense(4, activation='softmax', name='output')(x)

# Create the model
model2 = Model(inputs=[image_input, graph_input, adj_input], outputs=output)

# Compile the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model2.summary()


Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 flatten_10 (Flatten)           (None, 50176)        0           ['image_input[0][0]']            
                                                                                                  
 graph_input (InputLayer)       [(None, 64)]         0           []                               
                                                                                                  
 adj_input (InputLayer)         [(None, 5121)]       0           []                        

In [41]:
import numpy as np

# Pad the test adjacency matrix with zeros to match the shape of the train adjacency matrix
max_nodes = train_adj_matrix.shape[1]
test_adj_matrix_padded = np.pad(test_adj_matrix, ((0, 0), (0, max_nodes - test_adj_matrix.shape[1])), mode='constant')

# Verify the shape of the padded test adjacency matrix
print("Shape of padded test_adj_matrix:", test_adj_matrix_padded.shape)


Shape of padded test_adj_matrix: (1279, 5121)


In [42]:
# Define the number of epochs and batch size
epochs = 10
batch_size = 32

# Fit the model to the training data
history = model2.fit(
    {'image_input': alz_images_train_with_grayscale_index, 'graph_input': train_features, 'adj_input': train_adj_matrix},
    {'output': alz_labels_train_onehot},
    epochs=epochs,
    batch_size=batch_size,
    validation_data=({'image_input': alz_images_test_with_grayscale_index, 'graph_input': test_features, 'adj_input': test_adj_matrix_padded}, {'output': alz_labels_test_onehot})
)



Epoch 1/10
161/161 [==============================] - 16s 95ms/step - loss: 2.8730 - accuracy: 0.4933 - val_loss: 4.3943 - val_accuracy: 0.3503
Epoch 2/10
161/161 [==============================] - 15s 91ms/step - loss: 1.1173 - accuracy: 0.6171 - val_loss: 4.1000 - val_accuracy: 0.5004
Epoch 3/10
161/161 [==============================] - 15s 91ms/step - loss: 1.0209 - accuracy: 0.6469 - val_loss: 1.9302 - val_accuracy: 0.3972
Epoch 4/10
161/161 [==============================] - 15s 92ms/step - loss: 0.6391 - accuracy: 0.7424 - val_loss: 1.9182 - val_accuracy: 0.5387
Epoch 5/10
161/161 [==============================] - 15s 91ms/step - loss: 0.6414 - accuracy: 0.7442 - val_loss: 1.2701 - val_accuracy: 0.4910
Epoch 6/10
161/161 [==============================] - 15s 92ms/step - loss: 0.5399 - accuracy: 0.7727 - val_loss: 2.4503 - val_accuracy: 0.3886
Epoch 7/10
161/161 [==============================] - 14s 87ms/step - loss: 0.5566 - accuracy: 0.7834 - val_loss: 1.8231 - val_accuracy:

<h1>test cnn/graph</h1>

In [54]:
import numpy as np

# Pad the test adjacency matrix with zeros to match the shape of the train adjacency matrix
max_nodes = train_adj_matrix.shape[1]
test_adj_matrix_padded = np.pad(test_adj_matrix, ((0, 0), (0, max_nodes - test_adj_matrix.shape[1])), mode='constant')

# Verify the shape of the padded test adjacency matrix
print("Shape of padded test_adj_matrix:", test_adj_matrix_padded.shape)

Shape of padded test_adj_matrix: (1279, 5121)


In [84]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import regularizers

# Define input layers for image data and graph data
image_input = Input(shape=(img_height, img_width, 1), name='image_input')
graph_input = Input(shape=(train_features.shape[1],), name='graph_input')
adj_input = Input(shape=(train_adj_matrix.shape[1],), name='adj_input')

# Flatten the image data
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(image_input)
maxpool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(maxpool1)
maxpool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flatten_image = Flatten()(maxpool2)

# Concatenate flattened image data with graph data and adjacency matrix
concatenated_input = Concatenate()([flatten_image, graph_input, adj_input])

# Define the dense layers with dropout regularization
x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(concatenated_input)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)

# Output layer
output = Dense(4, activation='softmax', name='output')(x)

# Create the model
model_complex = Model(inputs=[image_input, graph_input, adj_input], outputs=output)

# Compile the model
model_complex.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# def exponential_decay(epoch, initial_lr=0.001, decay_rate=0.9):
#    return initial_lr * np.power(decay_rate, epoch)

# lr_scheduler_exp_decay = LearningRateScheduler(exponential_decay)

def cyclic_lr(epoch, lr_max=0.001, lr_min=0.0001, step_size=8):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = lr_min + (lr_max - lr_min) * np.maximum(0, (1 - x))
    return lr

lr_scheduler_cyclic_lr = LearningRateScheduler(cyclic_lr)

# Print model summary
#model_complex.summary()


In [85]:
# Fit the model to the training data
history = model_complex.fit(
    {'image_input': alz_images_train_with_grayscale_index, 'graph_input': train_features, 'adj_input': train_adj_matrix},
    {'output': alz_labels_train_onehot},
    epochs=30,
    batch_size=32,
    validation_data=({'image_input': alz_images_test_with_grayscale_index, 'graph_input': test_features, 'adj_input': test_adj_matrix_padded}, {'output': alz_labels_test_onehot}),
    callbacks=[lr_scheduler_cyclic_lr, early_stopping]
)

Epoch 1/30
161/161 [==============================] - 78s 484ms/step - loss: 2.3077 - accuracy: 0.4983 - val_loss: 1.8118 - val_accuracy: 0.5340 - lr: 1.0000e-04
Epoch 2/30
161/161 [==============================] - 79s 491ms/step - loss: 1.6609 - accuracy: 0.5456 - val_loss: 1.6519 - val_accuracy: 0.5059 - lr: 1.0000e-04
Epoch 3/30
161/161 [==============================] - 84s 525ms/step - loss: 1.4855 - accuracy: 0.5866 - val_loss: 1.5488 - val_accuracy: 0.5223 - lr: 1.0000e-04
Epoch 4/30
161/161 [==============================] - 78s 487ms/step - loss: 1.3716 - accuracy: 0.6116 - val_loss: 1.6318 - val_accuracy: 0.5012 - lr: 1.0000e-04
Epoch 5/30
161/161 [==============================] - 79s 489ms/step - loss: 1.2668 - accuracy: 0.6368 - val_loss: 1.4206 - val_accuracy: 0.5450 - lr: 1.0000e-04
Epoch 6/30
161/161 [==============================] - 78s 485ms/step - loss: 1.1985 - accuracy: 0.6714 - val_loss: 1.4485 - val_accuracy: 0.4996 - lr: 1.0000e-04
Epoch 7/30
161/161 [========

In [86]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import regularizers

# Define input layers for image data and graph data
image_input = Input(shape=(img_height, img_width, 1), name='image_input')
graph_input = Input(shape=(train_features.shape[1],), name='graph_input')
adj_input = Input(shape=(train_adj_matrix.shape[1],), name='adj_input')

# Flatten the image data
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.1))(image_input)
maxpool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.1))(maxpool1)
maxpool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flatten_image = Flatten()(maxpool2)

# Concatenate flattened image data with graph data and adjacency matrix
concatenated_input = Concatenate()([flatten_image, graph_input, adj_input])

# Define the dense layers with dropout regularization
x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(concatenated_input)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)

# Output layer
output = Dense(4, activation='softmax', name='output')(x)

# Create the model
model_complex2 = Model(inputs=[image_input, graph_input, adj_input], outputs=output)

# Compile the model
model_complex2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# def exponential_decay(epoch, initial_lr=0.001, decay_rate=0.9):
#    return initial_lr * np.power(decay_rate, epoch)

# lr_scheduler_exp_decay = LearningRateScheduler(exponential_decay)

def cyclic_lr(epoch, lr_max=0.001, lr_min=0.0001, step_size=8):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = lr_min + (lr_max - lr_min) * np.maximum(0, (1 - x))
    return lr

lr_scheduler_cyclic_lr = LearningRateScheduler(cyclic_lr)

# Print model summary
#model_complex.summary()

In [87]:
# Fit the model to the training data
# Fit the model to the training data
history2 = model_complex2.fit(
    {'image_input': alz_images_train_with_grayscale_index, 'graph_input': train_features, 'adj_input': train_adj_matrix},
    {'output': alz_labels_train_onehot},
    epochs=30,
    batch_size=32,
    validation_data=({'image_input': alz_images_test_with_grayscale_index, 'graph_input': test_features, 'adj_input': test_adj_matrix_padded}, {'output': alz_labels_test_onehot}),
    callbacks=[lr_scheduler_cyclic_lr, early_stopping]
)


Epoch 1/30
161/161 [==============================] - 82s 507ms/step - loss: 6.1124 - accuracy: 0.4812 - val_loss: 4.6646 - val_accuracy: 0.5145 - lr: 1.0000e-04
Epoch 2/30
161/161 [==============================] - 80s 498ms/step - loss: 3.9297 - accuracy: 0.5075 - val_loss: 3.2770 - val_accuracy: 0.5262 - lr: 1.0000e-04
Epoch 3/30
161/161 [==============================] - 80s 497ms/step - loss: 2.8328 - accuracy: 0.5155 - val_loss: 2.4607 - val_accuracy: 0.5106 - lr: 1.0000e-04
Epoch 4/30
161/161 [==============================] - 80s 499ms/step - loss: 2.1697 - accuracy: 0.5310 - val_loss: 1.9489 - val_accuracy: 0.5137 - lr: 1.0000e-04
Epoch 5/30
161/161 [==============================] - 79s 494ms/step - loss: 1.7794 - accuracy: 0.5298 - val_loss: 1.6577 - val_accuracy: 0.5223 - lr: 1.0000e-04
Epoch 6/30
161/161 [==============================] - 80s 498ms/step - loss: 1.5427 - accuracy: 0.5310 - val_loss: 1.4876 - val_accuracy: 0.5246 - lr: 1.0000e-04
Epoch 7/30
161/161 [========

In [90]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Flatten, Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import regularizers

# Define input layers for image data and graph data
image_input = Input(shape=(img_height, img_width, 1), name='image_input')
graph_input = Input(shape=(train_features.shape[1],), name='graph_input')
adj_input = Input(shape=(train_adj_matrix.shape[1],), name='adj_input')

# Flatten the image data
conv1 = Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, num_channels), 
           kernel_regularizer=regularizers.l2(0.1))(image_input)
conv1_bn = BatchNormalization()(conv1)
maxpool1 = MaxPooling2D(pool_size=(2, 2))(conv1_bn)


conv2 = Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.1))(maxpool1)
conv2_bn = BatchNormalization()(conv2)
maxpool2 = MaxPooling2D(pool_size=(2, 2))(conv2_bn)

#conv3 = (maxpool2)


flatten_image = Flatten()(maxpool2)

# Concatenate flattened image data with graph data and adjacency matrix
concatenated_input = Concatenate()([flatten_image, graph_input, adj_input])

# Define the dense layers with dropout regularization
x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(concatenated_input)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)

# Output layer
output = Dense(4, activation='softmax', name='output')(x)

# Create the model
model_complex_most = Model(inputs=[image_input, graph_input, adj_input], outputs=output)

# Compile the model
model_complex_most.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# def exponential_decay(epoch, initial_lr=0.001, decay_rate=0.9):
#    return initial_lr * np.power(decay_rate, epoch)

# lr_scheduler_exp_decay = LearningRateScheduler(exponential_decay)

def cyclic_lr(epoch, lr_max=0.001, lr_min=0.0001, step_size=8):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = lr_min + (lr_max - lr_min) * np.maximum(0, (1 - x))
    return lr

lr_scheduler_cyclic_lr = LearningRateScheduler(cyclic_lr)

In [91]:
history2 = model_complex_most.fit(
    {'image_input': alz_images_train_with_grayscale_index, 'graph_input': train_features, 'adj_input': train_adj_matrix},
    {'output': alz_labels_train_onehot},
    epochs=30,
    batch_size=32,
    validation_data=({'image_input': alz_images_test_with_grayscale_index, 'graph_input': test_features, 'adj_input': test_adj_matrix_padded}, {'output': alz_labels_test_onehot}),
    callbacks=[lr_scheduler_cyclic_lr, early_stopping]
)

Epoch 1/30
161/161 [==============================] - 92s 568ms/step - loss: 8.4156 - accuracy: 0.4663 - val_loss: 6.7515 - val_accuracy: 0.5012 - lr: 1.0000e-04
Epoch 2/30
161/161 [==============================] - 90s 556ms/step - loss: 5.9594 - accuracy: 0.5165 - val_loss: 5.0918 - val_accuracy: 0.5043 - lr: 1.0000e-04
Epoch 3/30
161/161 [==============================] - 92s 572ms/step - loss: 4.6070 - accuracy: 0.5116 - val_loss: 4.1905 - val_accuracy: 0.5012 - lr: 1.0000e-04
Epoch 4/30
161/161 [==============================] - 94s 585ms/step - loss: 3.6977 - accuracy: 0.5145 - val_loss: 3.2918 - val_accuracy: 0.5324 - lr: 1.0000e-04
Epoch 5/30
161/161 [==============================] - 93s 578ms/step - loss: 3.0166 - accuracy: 0.5341 - val_loss: 2.8122 - val_accuracy: 0.5426 - lr: 1.0000e-04
Epoch 6/30
161/161 [==============================] - 94s 585ms/step - loss: 2.5671 - accuracy: 0.5495 - val_loss: 2.4503 - val_accuracy: 0.5434 - lr: 1.0000e-04
Epoch 7/30
161/161 [========

In [92]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Flatten, Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import regularizers

# Define input layers for image data and graph data
image_input = Input(shape=(img_height, img_width, 1), name='image_input')
graph_input = Input(shape=(train_features.shape[1],), name='graph_input')
adj_input = Input(shape=(train_adj_matrix.shape[1],), name='adj_input')

# Flatten the image data
conv1 = Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, num_channels), 
           kernel_regularizer=regularizers.l2(0.1))(image_input)
conv1_bn = BatchNormalization()(conv1)
maxpool1 = MaxPooling2D(pool_size=(2, 2))(conv1_bn)


conv2 = Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.1))(maxpool1)
conv2_bn = BatchNormalization()(conv2)
maxpool2 = MaxPooling2D(pool_size=(2, 2))(conv2_bn)

conv3 = Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.1))(maxpool2)
conv3_bn = BatchNormalization()(conv3)
maxpool3 = MaxPooling2D(pool_size=(2, 2))(conv3_bn)


flatten_image = Flatten()(maxpool2)

# Concatenate flattened image data with graph data and adjacency matrix
concatenated_input = Concatenate()([flatten_image, graph_input, adj_input])

# Define the dense layers with dropout regularization
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(concatenated_input)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)

# Output layer
output = Dense(4, activation='softmax', name='output')(x)

# Create the model
model_complex_most_hard = Model(inputs=[image_input, graph_input, adj_input], outputs=output)

# Compile the model
model_complex_most_hard.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# def exponential_decay(epoch, initial_lr=0.001, decay_rate=0.9):
#    return initial_lr * np.power(decay_rate, epoch)

# lr_scheduler_exp_decay = LearningRateScheduler(exponential_decay)

def cyclic_lr(epoch, lr_max=0.001, lr_min=0.0001, step_size=8):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = lr_min + (lr_max - lr_min) * np.maximum(0, (1 - x))
    return lr

lr_scheduler_cyclic_lr = LearningRateScheduler(cyclic_lr)

In [93]:
history2 = model_complex_most_hard.fit(
    {'image_input': alz_images_train_with_grayscale_index, 'graph_input': train_features, 'adj_input': train_adj_matrix},
    {'output': alz_labels_train_onehot},
    epochs=30,
    batch_size=32,
    validation_data=({'image_input': alz_images_test_with_grayscale_index, 'graph_input': test_features, 'adj_input': test_adj_matrix_padded}, {'output': alz_labels_test_onehot}),
    callbacks=[lr_scheduler_cyclic_lr, early_stopping]
)

Epoch 1/30
161/161 [==============================] - 119s 732ms/step - loss: 15.5428 - accuracy: 0.5110 - val_loss: 13.4707 - val_accuracy: 0.2995 - lr: 1.0000e-04
Epoch 2/30
161/161 [==============================] - 117s 729ms/step - loss: 10.2522 - accuracy: 0.5700 - val_loss: 11.5304 - val_accuracy: 0.0743 - lr: 1.0000e-04
Epoch 3/30
161/161 [==============================] - 117s 728ms/step - loss: 7.6901 - accuracy: 0.6206 - val_loss: 7.4679 - val_accuracy: 0.5020 - lr: 1.0000e-04
Epoch 4/30
161/161 [==============================] - 117s 729ms/step - loss: 5.9401 - accuracy: 0.6825 - val_loss: 5.6892 - val_accuracy: 0.5434 - lr: 1.0000e-04
Epoch 5/30
161/161 [==============================] - 117s 727ms/step - loss: 4.7437 - accuracy: 0.7053 - val_loss: 4.5911 - val_accuracy: 0.6020 - lr: 1.0000e-04
Epoch 6/30
161/161 [==============================] - 117s 726ms/step - loss: 3.8584 - accuracy: 0.7448 - val_loss: 4.2190 - val_accuracy: 0.5465 - lr: 1.0000e-04
Epoch 7/30
161/161

In [108]:
import numpy as np

# Determine the desired shape
desired_shape = (test_adj_matrix_padded.shape[0], 5121)

# Calculate the amount of padding needed
padding_width = desired_shape[1] - test_adj_matrix_padded.shape[1]

# Pad the matrix
padded_adj_matrix = np.pad(test_adj_matrix_padded, ((0, 0), (0, padding_width)), mode='constant', constant_values=0)




In [110]:
import numpy as np

# Resize the adjacency matrix to the desired shape
resized_adj_matrix = np.resize(test_adj_matrix_padded, (test_adj_matrix_padded.shape[0], 5121))


In [112]:
print("Shape of test_adj_matrix_padded:", test_adj_matrix_padded.shape)


Shape of test_adj_matrix_padded: (1279, 5121)


In [113]:
import numpy as np

# Transpose the adjacency matrix
transposed_adj_matrix = np.transpose(test_adj_matrix_padded)


In [114]:
from sklearn.metrics import classification_report

# Evaluate the model on the test data
# Evaluate the model on the test data
test_loss, test_accuracy = model_complex_most_hard.evaluate([alz_images_test_with_grayscale_index, test_features, transposed_adj_matrix], alz_labels_test_onehot)




# # Evaluate the model on the test data
# test_loss, test_accuracy = model_complex_most_hard.evaluate(
#     {'image_input': alz_images_test_with_grayscale_index, 'graph_input': test_features, 'adj_input': test_padded_adj_matrix2},
#     {'output': alz_labels_test_onehot}
# )


# Predict the test labels
y_pred = model_complex2.predict([alz_images_test_with_grayscale_index, test_features, test_adj_matrix])

# Convert predicted labels from one-hot encoded format to categorical labels
y_pred_categorical = np.argmax(y_pred, axis=1)
test_labels_categorical = np.argmax(alz_labels_test_onehot, axis=1)

# Generate classification report
report = classification_report(test_labels_categorical, y_pred_categorical)
print(report)

ValueError: Data cardinality is ambiguous:
  x sizes: 1279, 1279, 5121
  y sizes: 1279
Make sure all arrays contain the same number of samples.

<h1>Jay's Algorthm</h1>
SVM and KNN (K-Nearest Neighbors)

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
def flatten_images(images):
    return images.reshape(images.shape[0], -1)
# Load and preprocess the images
train_dir = "./Alzheimer_s Dataset/train"
test_dir = "./Alzheimer_s Dataset/test"
alz_images_train, alz_labels_train = load_images_from_directory(train_dir)
alz_images_test, alz_labels_test = load_images_from_directory(test_dir)
# Flatten image data for KNN compatability
alz_images_train_flat = flatten_images(alz_images_train)
alz_images_test_flat = flatten_images(alz_images_test)
# Initialize KNN model
knn = KNeighborsClassifier(n_neighbors=3)  # You can tune the number of neighbors here
# Fit KNN model on the training data
knn.fit(alz_images_train_flat, alz_labels_train)
# KNN model on test data
alz_labels_pred = knn.predict(alz_images_test_flat)
# accuracy score
accuracy = accuracy_score(alz_labels_test, alz_labels_pred)
print(f"Test Accuracy: {accuracy:.2f}")
# Classification report
from sklearn.metrics import classification_report
print(classification_report(alz_labels_test, alz_labels_pred))
# Initialize lists to store accuracies and k values
k_values = list(range(1, 15))  # Evaluating k from 1 to 14
accuracies = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(alz_images_train_flat, alz_labels_train)
    alz_labels_pred = knn.predict(alz_images_test_flat)
    accuracy = accuracy_score(alz_labels_test, alz_labels_pred)
    accuracies.append(accuracy)
    print(f"Accuracy for k={k}: {accuracy:.2f}")
# Plot results
plt.figure(figsize=(10, 6))
plt.plot(k_values, accuracies, marker='o')
plt.title('KNN Accuracy vs Number of Neighbors')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

<h1>Geoffrey's Algorithm</h1>
Random Forest and RNN